In [3]:
import os
from glob import glob
import pandas as pd
from functools import reduce
import cv2

In [4]:
#Load all txt files and store in a list

train_list = glob('C:/datasets/carla/train/labels/*.txt')
valid_list = glob('C:/datasets/carla/valid/labels/*.txt')

#cleaning data
train_list = list(map(lambda x: x.replace('\\','/'), train_list))
valid_list = list(map(lambda x: x.replace('\\','/'), valid_list))

In [23]:
train_list

['C:/datasets/carla/train/labels/0_jpg.rf.e5bf47d73a56039e5c78a4979d045c12.txt',
 'C:/datasets/carla/train/labels/1000_jpg.rf.05828540abd07a1d89b55d7ddf41dc94.txt',
 'C:/datasets/carla/train/labels/1001_jpg.rf.3078eaaaa558f556e13f9a1774a72937.txt',
 'C:/datasets/carla/train/labels/1002_jpg.rf.4b4af412c0bc7e14b6c9e4dcc6b34f6c.txt',
 'C:/datasets/carla/train/labels/1003_jpg.rf.c46d8059910c862e010744a8ce9aa017.txt',
 'C:/datasets/carla/train/labels/1004_jpg.rf.63842fc86f7d8b00134735cce3e8b572.txt',
 'C:/datasets/carla/train/labels/1005_jpg.rf.cd1c8a42ef3099490d195aa1b8f32ae8.txt',
 'C:/datasets/carla/train/labels/1006_jpg.rf.e6bb83d66c344f0680f6f843062f38f6.txt',
 'C:/datasets/carla/train/labels/1007_jpg.rf.7c2f3c173442be0866fb928bb223867f.txt',
 'C:/datasets/carla/train/labels/1008_jpg.rf.b5c5dfcb0916d351aba354ecbadbec60.txt',
 'C:/datasets/carla/train/labels/1009_jpg.rf.edcff9695ae0b15262e31f2379053770.txt',
 'C:/datasets/carla/train/labels/100_jpg.rf.22e2380b694338e2306c5f49cd1350bc.tx

In [36]:
def convert_list(label_list):
    yolo_list = []
    for file in label_list:
        with open(file, 'r') as f:
            lines = f.readlines() 
        for line in lines:
            line = line.strip().split(' ')
            label = line[0]
            x_center, y_center, width, height = line[1:]
            txt_name = file.split('/')[5]
            img_name = txt_name.split('.txt') 
            name = img_name[0] +'.jpg'
            yolo_label = [name, label, x_center, y_center, width, height]
            yolo_list.append(yolo_label)
    return yolo_list


'''
def convert_list(label_list):
    yolo_list = []
    count = 0
    for file in label_list:
        with open(file, 'r') as f:
            lines = f.readlines() 
        for line in lines:
            line = line.strip().split(' ')
            label = line[0]
            l_list = [0, 1, 9]
            x_center, y_center, width, height = line[1:]
            txt_name = file.split('/')[5]
            img_name = txt_name.split('.txt') 
            name = img_name[0] +'.jpg'
            if int(label) in l_list:
                if int(label) == 9:
                    label = '2'
                yolo_label = [name, label, x_center, y_center, width, height]
                yolo_list.append(yolo_label)       
    return  yolo_list
'''
yolo_train = convert_list(train_list)
yolo_valid = convert_list(valid_list)


print(len(yolo_train))
print(len(yolo_valid))



2877
363


In [37]:
train_df = pd.DataFrame(yolo_train, columns= ['filename','class_id','x_center', 'y_center', 'width', 'height'])
valid_df = pd.DataFrame(yolo_valid, columns= ['filename','class_id','x_center', 'y_center', 'width', 'height'])

In [38]:
train_df

,filename,class_id,x_center,y_center,width,height
0,0_jpg.rf.e5bf47d73a56039e5c78a4979d045c12.jpg,6,0.7609375,0.45625,0.078125,0.1
1,1000_jpg.rf.05828540abd07a1d89b55d7ddf41dc94.jpg,3,0.7203125,0.409375,0.03125,0.09791666666666667
2,1001_jpg.rf.3078eaaaa558f556e13f9a1774a72937.jpg,5,0.83203125,0.33157894736842103,0.0609375,0.2736842105263158
3,1002_jpg.rf.4b4af412c0bc7e14b6c9e4dcc6b34f6c.jpg,9,0.57421875,0.7039473684210527,0.2796875,0.35526315789473684
4,1002_jpg.rf.4b4af412c0bc7e14b6c9e4dcc6b34f6c.jpg,9,0.27421875,0.5328947368421053,0.1078125,0.05
...,...,...,...,...,...,...
2872,9_jpg.rf.8973556a5e0ee8ad95b1ccdc8cb0dad9.jpg,3,0.58125,0.46770833333333334,0.0125,0.03958333333333333
2873,9_jpg.rf.8973556a5e0ee8ad95b1ccdc8cb0dad9.jpg,9,0.14453125,0.471875,0.2828125,0.8604166666666667
2874,9_jpg.rf.8973556a5e0ee8ad95b1ccdc8cb0dad9.jpg,9,0.92265625,0.528125,0.1515625,0.09375
2875,9_jpg.rf.8973556a5e0ee8ad95b1ccdc8cb0dad9.jpg,9,0.5,0.5322916666666667,0.053125,0.05625


In [47]:


# List of columns to convert




# Get the image width and height
image_width = 640
image_height = 480

# Example DataFrame
train_df = pd.DataFrame(yolo_train, columns=['filename', 'class_id', 'x_center', 'y_center', 'width', 'height'])
valid_df = pd.DataFrame(yolo_valid, columns= ['filename','class_id','x_center', 'y_center', 'width', 'height'])


# List of columns to convert
columns_to_convert = ['x_center', 'y_center', 'width', 'height']

# Convert specified columns to integer type for train_df
train_df[columns_to_convert] = train_df[columns_to_convert].astype(float)

# Convert specified columns to integer type for valid_df
valid_df[columns_to_convert] = valid_df[columns_to_convert].astype(float)

'''
train_df['x_center'] = train_df['x_center'].astype(float)
train_df['y_center'] = train_df['y_center'].astype(float)
train_df['width'] = train_df['width'].astype(float)
train_df['height'] = train_df['height'].astype(float)

valid_df['x_center'] = train_df['x_center'].astype(float)
valid_df['y_center'] = train_df['y_center'].astype(float)
valid_df['width'] = train_df['width'].astype(float)
valid_df['height'] = train_df['height'].astype(float)
'''


# Function to convert normalized coordinates to pixel values
def convert_coordinates(row, image_width, image_height):
    xmin = (row['x_center'] - row['width'] / 2) * image_width
    xmax = (row['x_center'] + row['width'] / 2) * image_width
    ymin = (row['y_center'] - row['height'] / 2) * image_height
    ymax = (row['y_center'] + row['height'] / 2) * image_height
    return xmin, xmax, ymin, ymax

# Convert normalized coordinates to pixel values for train DataFrame
train_df['xmin'], train_df['xmax'], train_df['ymin'], train_df['ymax'] = zip(*train_df.apply(lambda row: convert_coordinates(row, image_width, image_height), axis=1))

# Convert normalized coordinates to pixel values for valid DataFrame
valid_df['xmin'], valid_df['xmax'], valid_df['ymin'], valid_df['ymax'] = zip(*valid_df.apply(lambda row: convert_coordinates(row, image_width, image_height), axis=1))
   


In [48]:
train_df

,filename,class_id,x_center,y_center,width,height,xmin,xmax,ymin,ymax
0,0_jpg.rf.e5bf47d73a56039e5c78a4979d045c12.jpg,6,0.760938,0.456250,0.078125,0.100000,462.0,512.0,195.000000,243.000000
1,1000_jpg.rf.05828540abd07a1d89b55d7ddf41dc94.jpg,3,0.720313,0.409375,0.031250,0.097917,451.0,471.0,173.000000,220.000000
2,1001_jpg.rf.3078eaaaa558f556e13f9a1774a72937.jpg,5,0.832031,0.331579,0.060937,0.273684,513.0,552.0,93.473684,224.842105
3,1002_jpg.rf.4b4af412c0bc7e14b6c9e4dcc6b34f6c.jpg,9,0.574219,0.703947,0.279687,0.355263,278.0,457.0,252.631579,423.157895
4,1002_jpg.rf.4b4af412c0bc7e14b6c9e4dcc6b34f6c.jpg,9,0.274219,0.532895,0.107813,0.050000,141.0,210.0,243.789474,267.789474
...,...,...,...,...,...,...,...,...,...,...
2872,9_jpg.rf.8973556a5e0ee8ad95b1ccdc8cb0dad9.jpg,3,0.581250,0.467708,0.012500,0.039583,368.0,376.0,215.000000,234.000000
2873,9_jpg.rf.8973556a5e0ee8ad95b1ccdc8cb0dad9.jpg,9,0.144531,0.471875,0.282813,0.860417,2.0,183.0,20.000000,433.000000
2874,9_jpg.rf.8973556a5e0ee8ad95b1ccdc8cb0dad9.jpg,9,0.922656,0.528125,0.151562,0.093750,542.0,639.0,231.000000,276.000000
2875,9_jpg.rf.8973556a5e0ee8ad95b1ccdc8cb0dad9.jpg,9,0.500000,0.532292,0.053125,0.056250,303.0,337.0,242.000000,269.000000


In [50]:
train_df.groupby('filename')

train_df

,filename,class_id,x_center,y_center,width,height,xmin,xmax,ymin,ymax
0,0_jpg.rf.e5bf47d73a56039e5c78a4979d045c12.jpg,6,0.760938,0.456250,0.078125,0.100000,462.0,512.0,195.000000,243.000000
1,1000_jpg.rf.05828540abd07a1d89b55d7ddf41dc94.jpg,3,0.720313,0.409375,0.031250,0.097917,451.0,471.0,173.000000,220.000000
2,1001_jpg.rf.3078eaaaa558f556e13f9a1774a72937.jpg,5,0.832031,0.331579,0.060937,0.273684,513.0,552.0,93.473684,224.842105
3,1002_jpg.rf.4b4af412c0bc7e14b6c9e4dcc6b34f6c.jpg,9,0.574219,0.703947,0.279687,0.355263,278.0,457.0,252.631579,423.157895
4,1002_jpg.rf.4b4af412c0bc7e14b6c9e4dcc6b34f6c.jpg,9,0.274219,0.532895,0.107813,0.050000,141.0,210.0,243.789474,267.789474
...,...,...,...,...,...,...,...,...,...,...
2872,9_jpg.rf.8973556a5e0ee8ad95b1ccdc8cb0dad9.jpg,3,0.581250,0.467708,0.012500,0.039583,368.0,376.0,215.000000,234.000000
2873,9_jpg.rf.8973556a5e0ee8ad95b1ccdc8cb0dad9.jpg,9,0.144531,0.471875,0.282813,0.860417,2.0,183.0,20.000000,433.000000
2874,9_jpg.rf.8973556a5e0ee8ad95b1ccdc8cb0dad9.jpg,9,0.922656,0.528125,0.151562,0.093750,542.0,639.0,231.000000,276.000000
2875,9_jpg.rf.8973556a5e0ee8ad95b1ccdc8cb0dad9.jpg,9,0.500000,0.532292,0.053125,0.056250,303.0,337.0,242.000000,269.000000


In [46]:
valid_df

,filename,class_id,x_center,y_center,width,height,xmin,xmax,ymin,ymax
0,0_jpg.rf.4ab258123cdb46c01a93b6a6cd21d6df.jpg,3,0.914062,0.275000,0.084375,0.328947,558.0,612.0,53.052632,210.947368
1,101_jpg.rf.65da10a2f186be26cfa80c02d1f9c259.jpg,8,0.912500,0.438542,0.128125,0.160417,543.0,625.0,172.000000,249.000000
2,103_jpg.rf.f3b0bc74d3bdd7b844d404aa5ba65e08.jpg,3,0.602344,0.462500,0.010937,0.041667,382.0,389.0,212.000000,232.000000
3,103_jpg.rf.f3b0bc74d3bdd7b844d404aa5ba65e08.jpg,9,0.511719,0.536458,0.060937,0.060417,308.0,347.0,243.000000,272.000000
4,103_jpg.rf.f3b0bc74d3bdd7b844d404aa5ba65e08.jpg,9,0.389844,0.540625,0.051562,0.068750,233.0,266.0,243.000000,276.000000
...,...,...,...,...,...,...,...,...,...,...
358,97_jpg.rf.d0b46a16d8ed13b69d68d063b4f309c7.jpg,5,0.670312,0.421875,0.025000,0.072917,421.0,437.0,185.000000,220.000000
359,97_jpg.rf.d0b46a16d8ed13b69d68d063b4f309c7.jpg,2,0.547656,0.557292,0.045312,0.122917,336.0,365.0,238.000000,297.000000
360,98_jpg.rf.fb37528f11f2233af05905d0bb44b34d.jpg,9,0.586719,0.506579,0.032813,0.028947,365.0,386.0,236.210526,250.105263
361,9_jpg.rf.798851f4317f5185bf8d91c513c9a182.jpg,3,0.592187,0.462500,0.015625,0.045833,374.0,384.0,211.000000,233.000000


In [51]:
train_images = train_df['filename'].unique()
valid_images = valid_df['filename'].unique()
print(len(train_images),len(valid_images))

1520 195


In [53]:
import os
from shutil import copy
train = 'C:/datasets/carla/train/'
valid = 'C:/datasets/carla/valid/'

groupby_obj_train = train_df.groupby('filename')
groupby_obj_valid = valid_df.groupby('filename')

def save_data(filename, src_path, folder_path, group_obj):
    # move image
    src = os.path.join(src_path,filename)
    dst = os.path.join(folder_path,filename)
    copy(src,dst) # move image to the destination folder
    
    # save the labels
    text_filename = os.path.join(folder_path,
                                 os.path.splitext(filename)[0]+'.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename,sep=' ',index=False,header=False)

train_path = 'C:/datasets/carla/train/images'
valid_path = 'C:/datasets/carla/valid/images'

filename_series_train = pd.Series(groupby_obj_train.groups.keys())
filename_series_valid = pd.Series(groupby_obj_valid.groups.keys())

In [54]:
filename_series_train.apply(save_data,args=(train_path, train,groupby_obj_train))

0       None
1       None
2       None
3       None
4       None
        ... 
1515    None
1516    None
1517    None
1518    None
1519    None
Length: 1520, dtype: object

In [55]:
filename_series_valid.apply(save_data,args=(valid_path, valid,groupby_obj_valid))

0      None
1      None
2      None
3      None
4      None
       ... 
190    None
191    None
192    None
193    None
194    None
Length: 195, dtype: object

In [56]:
train_df.to_csv('C:/datasets/carla/train/carla_train.csv', index=False)

In [57]:
valid_df.to_csv('C:/datasets/carla/train/carla_valid.csv', index=False)